# Image Registration and Combination

## Author
[C. E. Brasseur](https://ceb8.github.io/)

## Learning Goals
* Correcting image World Coordinate System (WCS) mapping
* Aligning one image to another
* Co-adding images


## Keywords
FITS, WCS, DaoFind, ccdproc, source extraction, wcs fitting, reproject, image alignment, coordinate crossmatch


## Summary
Often we want to combine images, either by co-adding to improve signal-to-noise, or subtracting to bring out variable stars, or combining different wavebands to colorize an image.  In this tutorial I will lead you through this entire process, starting with two images from the Las Cumbres Observatory.

First we must correct the pixel-to-world coordinate mapping, for which we will crossmatch sources in the image against the Gaia catalog. These the images then must be projected onto the same alignement, which we do using `reproject`. Finally we use `ccdproc` to combine the images.

I assume the learner is comfortable using FITS files and WCS objects (see the [FITS-images](https://learn.astropy.org/tutorials/FITS-images.html) and [FITS-cubes](https://learn.astropy.org/tutorials/FITS-cubes.html) tutorials).


## Table of Contents

- [1. Imports](#1.-Imports)
- [2. Download the data](#2.-Download-the-data)
- [3. Correct the WCS info](#3.-Correct-the-WCS-info)
- [4. Image Registration](#4.-Image-Registration)
- [5. Coadd the images](#5.-Coadd-the-images)
- [6. Save stacked image](#6.-Save-stacked-image)


## 1. Imports

In [ ]:
import numpy as np

# For downloading observations
import requests
import json

# For Querying Gaia
from astroquery.mast import Catalogs

# For source finding
from photutils.detection import DAOStarFinder

# For WCS fitting
from astropy.wcs.utils import fit_wcs_from_points

# For image alignment
from reproject import reproject_interp

# For image co-adding
from ccdproc import CCDData, Combiner

# For interaction with FITS image files
from astropy.io import fits
from astropy.wcs import WCS

# Other useful astropy imports
from astropy.stats import sigma_clipped_stats
from astropy.table import Table
from astropy.coordinates import SkyCoord
import astropy.units as u

# For plotting
import matplotlib
%matplotlib inline

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

## 2. Download the data

For this tutorial we will use two images taken by the Las Cumbres Observatory in 2015. We will download two images of the globular cluster NGC 1866 in the V-band. These images can be searched for at the [LCO Archive](https://archive.lco.global/), however for this tutorial we will assume we already know the files we want.

In [ ]:
def get_lco_observation(frame):
    """
    Given a frame value download the associated LCO obvservation fits file.
    """
    
    frame_url = f"https://archive-api.lco.global/frames/{frame}"
    result = requests.get(frame_url)
    
    file_info = json.loads(result.content)
    data = requests.get(file_info["url"])
    
    with open(file_info["filename"], 'wb') as FLE:
        FLE.write(data.content)
        
    return file_info["filename"]

In [ ]:
file_1 = get_lco_observation(915222)
file_2 = get_lco_observation(934536)

### Plotting the data

Here we open both files and plot them individually and together. We can see from the composite image that the two images are offset from each other, so we will need to align them befor we can co-add them.

In [ ]:
hdu_1 = fits.open(file_1)
img_1 = hdu_1[1].data
hdr_1 = hdu_1[1].header
hdu_1.close()

hdu_2 = fits.open(file_2)
img_2 = hdu_2[1].data
hdr_2 = hdu_2[1].header
hdu_2.close()

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(12,4))

for ax in axs:
    ax.set_axis_off()

axs[0].imshow(img_1, cmap='Oranges_r', norm=LogNorm(vmin=10,vmax=60))
axs[2].imshow(img_1, cmap='Oranges_r', norm=LogNorm(vmin=10,vmax=60), alpha=0.5)

axs[1].imshow(img_2, cmap='Blues_r', norm=LogNorm(vmin=10,vmax=60))
axs[2].imshow(img_2, cmap='Blues_r', norm=LogNorm(vmin=10,vmax=60), alpha=0.5)

plt.show()

## 3. Correct the WCS info

Before we can align the images we need to make sure they both have correct WCS information.The observation files include WCS information, however it is not accurate enough for our purposes.

We will first focus on the process for a single file (`hdr_1` and `img_1`).

To do this we need to identify a handful of bright sources in our image (x,y pixel coordinates) and match them to canonical world coordinates (ra,dec) for those sources. We can then use the list of x,y and canonical ra,dec pairs to build the corrected WCS object.

**Note:** Loading the WCS information from the observation header gives a warning. This is not a problem, it is astropy telling us it has "fixed" the header WCS keywords to make them conform to the official standards.

In [ ]:
init_wcs = WCS(hdr_1)
print(init_wcs)

### a. Finding source locations in the image.

We will use `DAOStarFinder` (from the [detection module](https://photutils.readthedocs.io/en/stable/detection.html) in photutils) to identify sources within our image. We estimate the background level (median) and background noise (std)using [sigma-clipped statistics](https://docs.astropy.org/en/stable/api/astropy.stats.sigma_clipped_stats.html).

In [ ]:
mean, median, std = sigma_clipped_stats(img_1, sigma=3.0)    
print(f"Image mean: {mean:.1f}\nImage median: {median:.1f}\nImage std: {std:.1f}") 

We now create a `DAOStarFinder` looking for stars with a FWHM of ~6 px and peaks at least 120 standard deviations above the background. We set the threshold so high because we want to use only a handful of the brightest stars for our reference objects.

In [ ]:
daofind = DAOStarFinder(fwhm=6, threshold=120*std) # We only want the brightest sources
sources = daofind(img_1 - median)

### b. Gaia sources

We will use the [Gaia Catalog](https://gea.esac.esa.int/archive/) through [astroquery](https://astroquery.readthedocs.io/en/latest/mast/mast.html#catalog-queries) to get the canonical sky coordinates for our reference stars.

In [ ]:
# Pull out the center coordinate of the image
center_ra,center_dec = init_wcs.wcs.crval
coord = SkyCoord(ra=center_ra, dec=center_dec, unit=(u.degree, u.degree), frame='icrs')

# Performing the query
gaia_catalog = Catalogs.query_region(coordinates=coord, radius=0.095*u.deg, catalog="Gaia")

In [ ]:
# Filtering down to brightest sources not in the dense center

gaia_bright = gaia_catalog[gaia_catalog["phot_g_mean_mag"] < 15]
gaia_bright = gaia_bright[gaia_bright['distance'] > 3]

gaia_bright['coord'] = SkyCoord(gaia_bright['ra'], gaia_bright['dec'], unit=u.deg)

In [ ]:
# Using the observation WCS to get the image coordinates for the Gaia sources

pxs = init_wcs.world_to_pixel(gaia_bright['coord'])
gaia_bright["x"] = pxs[0]
gaia_bright["y"] = pxs[1]

### c. Plotting image and Gaia sources together

Before we embark on correcting the image WCS object we plot both the image sources from `DAOStarFinder`, and the canonical sources from Gaia. We can see that there are a few image sources (blue) without Gaia counterparts (red), and that the Gaia sources are offset from the image sources. The offset is due to errors in the image WCS, which is what we are aiming to correct.

In [ ]:
fig, ax = plt.subplots(figsize=(7,7))
ax.set_axis_off()

ax.imshow(img_1, cmap='gray', norm=LogNorm(vmin=10,vmax=60))

ax.scatter(sources['xcentroid'], sources['ycentroid'], ec='#44aeed', fc="none", s=150, lw=2, label="Image source")
ax.scatter(gaia_bright["x"], gaia_bright["y"], ec='#e71f71', fc="none", s=50, lw=2, label="Gaia source")

ax.legend(fontsize=13)

plt.show()

### d. Catalog crossmatch

Now that we have our reference sources from the image and Gaia, we need to pair the two lists properly. To do this we will crossmatch the list of bright Gaia sources with the list of image sources using [`match_to_catalog_sky`](https://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html#astropy.coordinates.SkyCoord.match_to_catalog_sky). 



In [ ]:
# Getting sky coordinates for image sources
sources["coord"] = init_wcs.pixel_to_world(sources['xcentroid'], sources['ycentroid'])

# Performing the cross match
idx, d2d, d3d = gaia_bright['coord'].match_to_catalog_sky(sources["coord"])

`match_to_catalog_sky` returns the indexes of the input catalog (`sources`) that correspondes to matched source in base catalog (`gaia_bright`), and the 2- and 3-D distances between the sources. We use that indesing to update the pixel positions of the `gaia_bright` sources, and add a column with the 2D distances, which are the calculated offsets between the gaia source pixel positions and the true image pixel positions.

In [ ]:
# Update the x/y columns in the gaia catalog
gaia_bright["x"] = sources['xcentroid'][idx]
gaia_bright["y"] = sources['ycentroid'][idx]

# Add the distance column (we have no distances so no need for the 3d distances)
gaia_bright["d2d"] = d2d.deg

print("Nearest Source Distances:")
for row in gaia_bright:
    print(f"{row['d2d']:.3f} deg")

We print out the distances to make sure we're indeed connecting the correct catalog entries. All of them look good (i.e. small and similar to each other), so we can move on to use [`fit_wcs_from_points`](https://docs.astropy.org/en/stable/api/astropy.wcs.utils.fit_wcs_from_points.html) to build the corrected WCS.

In [ ]:
corrected_wcs_1 = fit_wcs_from_points([gaia_bright["x"],gaia_bright["y"]], gaia_bright["coord"])
print(corrected_wcs_1)

Now we can update the Gaia x/y coordinates, using the corrected WCS, and plot the sources again. This time they line up correctly.

In [ ]:
pxs = corrected_wcs_1.world_to_pixel(gaia_bright['coord'])
gaia_bright["x"] = pxs[0]
gaia_bright["y"] = pxs[1]

In [ ]:
fig, ax = plt.subplots(figsize=(7,7))
ax.set_axis_off()

ax.imshow(img_1, cmap='gray', norm=LogNorm(vmin=10,vmax=60))

ax.scatter(sources['xcentroid'], sources['ycentroid'], ec='#44aeed', fc="none", s=150, lw=2, label="Image source")
ax.scatter(gaia_bright["x"], gaia_bright["y"], ec='#e71f71', fc="none", s=50, lw=2, label="Gaia source")

ax.legend(fontsize=13)

plt.show()

### e. Doing the same for the other image

Next we need to go through all the same steps for the second image.

In [ ]:
# Finding source locations on the image

init_wcs = WCS(hdr_2)

mean, median, std = sigma_clipped_stats(img_2, sigma=3.0)    

daofind = DAOStarFinder(fwhm=6.2, threshold=120*std) # We only want the brightest sources
sources = daofind(img_2 - median)

In [ ]:
# Using the observation WCS to get the image coordinates for the Gaia sources

pxs = init_wcs.world_to_pixel(gaia_bright['coord'])
gaia_bright["x"] = pxs[0]
gaia_bright["y"] = pxs[1]

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
ax.set_axis_off()

ax.imshow(img_2, cmap='gray', norm=LogNorm(vmin=10,vmax=60))

ax.scatter(sources['xcentroid'], sources['ycentroid'], ec='#44aeed', fc="none", s=150, lw=2, label="Image source")
ax.scatter(gaia_bright["x"], gaia_bright["y"], ec='#e71f71', fc="none", s=50, lw=2, label="Gaia source")

ax.legend(fontsize=13)

plt.show()

In [ ]:
# Catalog crossmatch
sources["coord"] = init_wcs.pixel_to_world(sources['xcentroid'], sources['ycentroid'])

idx, d2d, d3d = gaia_bright['coord'].match_to_catalog_sky(sources["coord"])

gaia_bright["x"] = sources['xcentroid'][idx]
gaia_bright["y"] = sources['ycentroid'][idx]
gaia_bright["d2d"] = d2d.deg

In [ ]:
# Correcting the WCS
corrected_wcs_2 = fit_wcs_from_points([gaia_bright["x"],gaia_bright["y"]], gaia_bright["coord"])

pxs = corrected_wcs_2.world_to_pixel(gaia_bright['coord'])
gaia_bright["x"] = pxs[0]
gaia_bright["y"] = pxs[1]

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
ax.set_axis_off()

ax.imshow(img_2, cmap='gray', norm=LogNorm(vmin=10,vmax=60))

ax.scatter(sources['xcentroid'], sources['ycentroid'], ec='#44aeed', fc="none", s=150, lw=2, label="Image source")
ax.scatter(gaia_bright["x"], gaia_bright["y"], ec='#e71f71', fc="none", s=50, lw=2, label="Gaia source")

ax.legend(fontsize=13)

plt.show()

## 4. Image Registration

At this point we have two images, each with a WCS object that we know is correct and consistent. The next step is to use the [`reproject`](https://reproject.readthedocs.io/en/stable/) package to register the images together. 


We will align `img_2` onto `img_1`. This means that we will interpolate `img_2` onto the pixel grid of `img_1`.

In [ ]:
img_2_aligned, footprint = reproject_interp((img_2, corrected_wcs_2), corrected_wcs_1, shape_out=img_2.shape)

Plotting the images together, we can see they are no longer offset. 

Note also the orange border on the left and top. This shows us how `img_2` had to be shifted to align with `img_1`.

In [ ]:
fig, ax = plt.subplots(figsize=(7,7))
ax.set_axis_off()

ax.imshow(img_1, cmap='Oranges_r', norm=LogNorm(vmin=10,vmax=60), alpha=0.5)
ax.imshow(img_2_aligned, cmap='Blues_r', norm=LogNorm(vmin=10,vmax=60), alpha=0.5)

plt.show()

## 5. Coadd the images

A common reason to align images is to allow the user to coadd the images to increase signal-to-noise and decrease the background noise. Here we will use the [image combination](https://ccdproc.readthedocs.io/en/latest/image_combination.html) functionality from [ccdproc](https://ccdproc.readthedocs.io/en/latest/index.html), which offers a variety of useful options. 

In [ ]:
# Load each image into a CCDData frame

ccd_frame_1 = CCDData(img_1, unit=u.dimensionless_unscaled)
ccd_frame_2 = CCDData(img_2_aligned, unit=u.dimensionless_unscaled)

In [ ]:
# Build the image combiner

combiner = Combiner([ccd_frame_1, ccd_frame_2])
combiner.data_arr.mask[np.isnan(combiner.data_arr)]=True

We produce a combined image by averaging the input images.

In [ ]:
coadd_img = combiner.average_combine()

In [ ]:
fig, ax = plt.subplots(figsize=(7,7))
ax.set_axis_off()

ax.imshow(coadd_img, cmap='gray', norm=LogNorm(vmin=10,vmax=60))

plt.show()

We can take the mediane of the input images instead.

In [ ]:
coadd_img = combiner.median_combine()

In [ ]:
fig, ax = plt.subplots(figsize=(7,7))
ax.set_axis_off()

ax.imshow(coadd_img, cmap='gray', norm=LogNorm(vmin=10,vmax=60))

plt.show()

## 6. Save stacked image

Now that we've processed our image, we can save the result to a new FITS file. 

In [ ]:
hdu = fits.PrimaryHDU(coadd_img, header=corrected_wcs_1.to_header())
hdu.writeto("combined_img.fits")